In [1]:
!pip install basicsr
!pip install realesrgan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 23.0 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214818 sha256=e0969a2ec5e458bbbf4ff64f88bae9115da086fcb5a2f7a610db18fbdbd4145e
  Stored in directory: /root/.cache/pip/wheels/38/83/99/2d8437cc652a01af27df5ff037a4075e95b52d67705c5f30ca
Successfully built basicsr
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.9 MB/s eta 

In [3]:
import os
import torch
from torchvision import datasets, transforms
from PIL import Image
import numpy as np
import cv2
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
from tqdm import tqdm

In [4]:
# Define transformations and load CIFAR-10 dataset
transform = transforms.ToTensor()
cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:04<00:00, 42.6MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# Define Real-ESRGAN Upscaler
model_path = '/content/drive/MyDrive/Colab Notebooks/Machine Learning Lab/Project/RealESRGAN_x4plus.pth'  # Path to your pre-trained 8x model
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
# Initialize RRDBNet model
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=8)
upscaler = RealESRGANer(scale=8, model_path=model_path, model=model, tile=0, tile_pad=10, pre_pad=0, half=True, device=device)

/usr/local/lib/python3.10/dist-packages/realesrgan/utils.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loadnet = torch.load(model_path, map_location=torch.device('cpu

In [7]:
# Define output directories
train_output_dir = '/content/drive/MyDrive/Colab Notebooks/Machine Learning Lab/Project/CIFAR10_train'
test_output_dir = '/content/drive/MyDrive/Colab Notebooks/Machine Learning Lab/Project/CIFAR10_test'
os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(test_output_dir, exist_ok=True)

In [8]:
def process_dataset(dataset, output_dir):
    """Process CIFAR-10 dataset and save enhanced images."""
    for idx, (image_tensor, label) in enumerate(tqdm(dataset, desc=f"Processing {output_dir}")):
        image = transforms.ToPILImage()(image_tensor)

        # Convert PIL image to OpenCV format (BGR)
        input_img = np.array(image)
        input_img_bgr = input_img[:, :, ::-1]  # Convert RGB to BGR for OpenCV

        # Enhance the image
        with torch.no_grad():
            try:
                output_img_bgr, _ = upscaler.enhance(input_img_bgr, outscale=8)
                output_img_bgr = cv2.resize(output_img_bgr, (224, 224), interpolation=cv2.INTER_LANCZOS4)
            except Exception as e:
                print(f"Error processing image {idx}: {e}")
                continue

        # Convert to RGB and save as PIL image
        output_img_rgb = output_img_bgr[:, :, ::-1]  # Convert BGR back to RGB
        output_image_pil = Image.fromarray(output_img_rgb)

        # Save the enhanced image in the corresponding label folder
        label_dir = os.path.join(output_dir, str(label))
        os.makedirs(label_dir, exist_ok=True)
        output_image_pil.save(os.path.join(label_dir, f"{idx}.png"))

# Process and save CIFAR-10 train and test datasets
process_dataset(cifar10_train, train_output_dir)
process_dataset(cifar10_test, test_output_dir)

print("Enhanced CIFAR-10 dataset creation complete!")

Processing /content/drive/MyDrive/Colab Notebooks/Machine Learning Lab/Project/CIFAR10_train: 100%|██████████| 50000/50000 [1:10:03<00:00, 11.89it/s]
Processing /content/drive/MyDrive/Colab Notebooks/Machine Learning Lab/Project/CIFAR10_test: 100%|██████████| 10000/10000 [14:06<00:00, 11.82it/s]

Enhanced CIFAR-10 dataset creation complete!
